In [1]:
import sys
import os
# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

import pandas as pd
import yaml
from datetime import datetime as dt, date
import requests
import logging
from datetime import datetime, timedelta
from src.common.AssetData import AssetData
from src.common.AssetDataService import AssetDataService
from src.common.AssetFileInOut import AssetFileInOut
from src.databaseService.Parser import Parser_AV
from src.databaseService.CleanData import CleanData
from src.common.AssetDataService import AssetDataService

from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData

import sys

logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(fmt="%(message)s")
handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(handler)
else:
    logger.handlers[:] = [handler]
# Usage
logger.info("This will print to the notebook’s output cell")

This will print to the notebook’s output cell


In [2]:
# Define paths
current_dir = os.getcwd()
desired_folder = "secrets"
absolute_path_to_folder = os.path.join(os.path.abspath(os.path.join(current_dir, "..")), "secrets")

# Path to the YAML file
yaml_file_path = os.path.join("../secrets", "alphaVantage.yaml")

# Read and load the YAML file
try:
    with open(yaml_file_path, 'r') as file:  # Open the YAML file for reading
        config = yaml.safe_load(file)  # Load the YAML content
        apiKey = config['alphaVantage_premium']['apiKey']  # Access the required key
except PermissionError:
    print("Permission denied. Please check file permissions.")
except FileNotFoundError:
    print("File not found. Please verify the path.")
except KeyError:
    print("KeyError: Check the structure of the YAML file.")
except yaml.YAMLError as e:
    print("YAML Error:", e)

In [11]:
ticker = "AXL"
ts = TimeSeries(key=apiKey, output_format='pandas')
fd = FundamentalData(key=apiKey, output_format='pandas')

datashareprice, _ = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
loadad_overview, _ = fd.get_company_overview(symbol=ticker)

url = 'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol='+ticker+'&apikey='+apiKey
incStatementData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=CASH_FLOW&symbol='+ticker+'&apikey='+apiKey
cashFlowData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol='+ticker+'&apikey='+apiKey
balanceSheetData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol='+ticker+'&apikey='+apiKey
earningsData = requests.get(url).json()

if incStatementData=={} or cashFlowData == {} or balanceSheetData == {} or earningsData == {}:
    raise ImportError(f"Empty Financial Data")

In [12]:
res = Parser_AV(sharepriceData=datashareprice).parse_shareprice()

print(res.tail(5))
print(res.dtypes)

            Date  Open    High     Low  Close  AdjClose     Volume  Dividends  \
6463  2025-07-15  4.85  4.9553   4.765   4.78      4.78  3104259.0        0.0   
6464  2025-07-16  4.82    4.86    4.71   4.85      4.85  3037769.0        0.0   
6465  2025-07-17  4.81    4.96    4.77   4.77      4.77  2806259.0        0.0   
6466  2025-07-18   4.8   4.825     4.5   4.55      4.55  2865667.0        0.0   
6467  2025-07-21  4.61    4.72  4.5241   4.53      4.53  2297598.0        0.0   

      Splits  
6463     1.0  
6464     1.0  
6465     1.0  
6466     1.0  
6467     1.0  
Date         string[python]
Open                Float64
High                Float64
Low                 Float64
Close               Float64
AdjClose            Float64
Volume              Float64
Dividends           Float64
Splits              Float64
dtype: object


In [13]:
loadad_about, loaded_sector = Parser_AV(overview=loadad_overview).parse_overview()

print("About: ", loadad_about)
print("Sector: ", loaded_sector)

About:  {'Symbol': 'AXL', 'AssetType': 'Common Stock', 'Name': 'American Axle & Manufacturing', 'Description': 'American Axle & Manufacturing Holdings, Inc. designs, designs and manufactures transmission line and metal forming products in the United States, Mexico, South America, China, other Asian countries, and Europe. The company is headquartered in Detroit, Michigan.', 'CIK': '1062231', 'Exchange': 'NYSE', 'Currency': 'USD', 'Country': 'USA', 'Sector': 'MANUFACTURING', 'Industry': 'MOTOR VEHICLE PARTS & ACCESSORIES', 'Address': 'ONE DAUCH DRIVE, DETROIT, MI, US', 'OfficialSite': 'https://www.aam.com', 'FiscalYearEnd': 'December', 'LatestQuarter': '2025-03-31', 'MarketCapitalization': '537539000', 'EBITDA': '717400000', 'PERatio': '25.17', 'PEGRatio': '-1.17', 'BookValue': '5.04', 'DividendPerShare': 'None', 'DividendYield': 'None', 'EPS': '0.18', 'RevenuePerShareTTM': '50.39', 'ProfitMargin': '0.0036', 'OperatingMarginTTM': '0.0432', 'ReturnOnAssetsTTM': '0.0301', 'ReturnOnEquityTT

In [14]:
financials_annually, financials_quarterly = Parser_AV(
    incStatementData = incStatementData, 
    cashFlowData = cashFlowData, 
    balanceSheetData = balanceSheetData, 
    earningsData = earningsData
).parse_financials()

In [15]:
print(financials_annually.tail(3))
print(financials_annually.dtypes)

   fiscalDateEnding  reportedEPS  grossProfit  totalRevenue         ebit  \
23       2022-12-31         0.61  704900000.0  5802400000.0  240800000.0   
24       2023-12-31        -0.09  624300000.0  6079500000.0  177200000.0   
25       2024-12-31         0.51  741400000.0  6124900000.0  251300000.0   

         ebitda   totalAssets  totalCurrentLiabilities  \
23  732900000.0  5469400000.0             1199300000.0   
24  664400000.0  5356300000.0             1201600000.0   
25  721000000.0  5059900000.0             1175200000.0   

    totalShareholderEquity  operatingCashflow  
23             627300000.0        448900000.0  
24             604900000.0        396100000.0  
25             562800000.0        455400000.0  
fiscalDateEnding           string[python]
reportedEPS                       Float64
grossProfit                       Float64
totalRevenue                      Float64
ebit                              Float64
ebitda                            Float64
totalAssets       

In [16]:
print(financials_quarterly.tail(3))
print(financials_quarterly.dtypes)

    fiscalDateEnding reportedDate  reportedEPS  estimatedEPS  surprise  \
102       2024-09-30   2024-11-08          0.2         -0.05      0.25   
103       2024-12-31   2025-02-14        -0.06       -0.1566    0.0966   
104       2025-03-31   2025-05-01         0.09        0.0649    0.0251   

     surprisePercentage  reportTime  grossProfit  totalRevenue        ebit  \
102               500.0  pre-market  171300000.0  1504900000.0  43100000.0   
103             61.6858  pre-market  154300000.0  1380800000.0  36900000.0   
104             38.6749  pre-market  173900000.0  1411300000.0        <NA>   

          ebitda   totalAssets  totalCurrentLiabilities  \
102  160000000.0  5328300000.0             1294200000.0   
103  152300000.0  5059900000.0             1175200000.0   
104   42700000.0  5139400000.0             1184100000.0   

     totalShareholderEquity  commonStockSharesOutstanding  operatingCashflow  
102             616300000.0                   122200000.0        143600000